# Glaciers Fronts

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# Uncomment if running in Google Colaboratory.
#!curl https://course.fast.ai/setup/colab | bash

In [ ]:
import os
from fastai.vision import *
from fastai.callbacks import *
from fastai.utils.mem import *

## Get Data

In [ ]:
fname = Path('glacier-fronts.tar.gz')
if not fname.exists():
    !wget -O {fname} https://www.dropbox.com/s/8dopo5hs9bzujgu/glacier-fronts.tar.gz?dl=0
    os.makedirs('data', exist_ok=True)
    !tar -xzf {fname} -C .

In [ ]:
path_data = Path('./data')
path_data.ls()

In [ ]:
path_trn = path_data/'train/images_equalize_autocontrast_smooth_edgeEnhance'
path_lbl = path_data/'train/masks_1px'
path_tst = path_data/'test/images_equalize_autocontrast_smooth_edgeEnhance'

In [ ]:
fnames = get_image_files(path_trn)
fnames[3]

In [ ]:
img_f = fnames[0]
img = open_image(img_f)
img.show(figsize=(10,10))

In [ ]:
get_y_fn = lambda x:  path_lbl/f'{x.name.replace("Subset","Front")}'

In [ ]:
mask = open_mask(get_y_fn(img_f))
mask.show(figsize=(10,10), alpha=1)

In [ ]:
src_size = np.array(mask.shape[1:])
src_size,mask.data

In [ ]:
classes = [0, 1]

## Create Dataset

In [ ]:
size = src_size

free = gpu_mem_get_free_no_cache()
# the max size of bs depends on the available GPU RAM
if free > 8200: bs=8
else:           bs=4
print(f"using bs={bs}, have {free}MB of GPU RAM free")

In [ ]:
data = (SegmentationItemList.from_folder(path_trn)
          .split_by_rand_pct(0.1)
          .label_from_func(get_y_fn, classes=classes)
          .transform(get_transforms(), size=size, tfm_y=True)
          .databunch(bs=bs)
          .normalize(imagenet_stats))

In [ ]:
data.show_batch(2, figsize=(10,10))

In [ ]:
data.classes

## Create Model

In [ ]:
def acc_glacier(input, target):
    target = target.squeeze(1)
    return (input.argmax(dim=1)==target).float().mean()

In [ ]:
metrics=acc_glacier
wd=1e-2
class_weights = Tensor([375.,1.]).cuda() # increase penalty on calving front pixels to handle class imbalance.
loss_func = CrossEntropyFlat(weight=class_weights,axis=1)

In [ ]:
learn = unet_learner(data, models.resnet34, loss_func=loss_func, metrics=metrics, wd=wd)

## Train

In [ ]:
lr_find(learn)
learn.recorder.plot()

In [ ]:
lr=3e-4

In [ ]:
learn.fit_one_cycle(10)

In [ ]:
learn.show_results(rows=2, figsize=(10,10))

## Test

In [ ]:
test = (ImageList.from_folder(path_tst))
img.show(y=learn.predict(img)[0])

## fin